<a href="https://colab.research.google.com/github/atreyee80/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backpor

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-29 04:27:58--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.21MB/s    in 0.8s    

2021-06-29 04:28:00 (1.21 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = ""
spark.sparkContext.addFile("https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz")
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

In [ ]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(df["review_id"],df["star_rating"],df["helpful_votes"],df["total_votes"],df["vine"],df["verified_purchase"])

In [ ]:
vine_df .show()
vine_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R3EQSTM9PWRAL|          3|            0|          0|   N|                Y|
| RBWPRK17XKIXD|          5|            0|          0|   N|                Y|
| RRSLOAF273XFC|          5|            1|          2|   N|                Y|
|R3S8W9Q6SWIT8O|          4|            0|          0|   N|                Y|
|R3QQ6NSLRVBFJC|          4|            0|          0|   N|                Y|
|R2XYDBMHUVJCSX|          2|            0|          1|   N|                Y|
|R1L94ESVC657A9|          5|            0|          0|   N|                Y|
|R3DI4B8LDWFQ3K|          1|            3|          4|   N|                Y|
|R1G7VV4HCXUQQL|          5|            0|          0|   N|                Y|
|R3TTNYN1TAMUSU|          5|            0|          0|   N|     

5331449

In [ ]:
filter_vine_df = vine_df.filter(vine_df["total_votes"] >= 20)
filter_vine_df.count()

133416

In [ ]:
filter_vine_df.show()
filter_vine_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
|R1M8EEGH2TVS0Q|          1|            6|         29|   N|                Y|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|     

133416

In [ ]:
filter_vine_df2 = filter_vine_df.filter(filter_vine_df["helpful_votes"]/filter_vine_df["total_votes"]>=.5)

In [ ]:
filter_vine_df2.show()
filter_vine_df2.count()



+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|                Y|
| RYA9ESIATW4EC|          1|           23|         27|   N|     

121360

In [ ]:
paid_review_yes = filter_vine_df2.filter(filter_vine_df2["vine"]=="Y")

In [ ]:
paid_review_yes.show()
paid_review_yes.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RG70K1HMY4LXX|          4|           37|         44|   Y|                N|
|R1DHGTNXDXJ0GB|          5|           78|         93|   Y|                N|
|R3B5S3OSA8N6AO|          5|           38|         43|   Y|                N|
|R29X0YBS0SBIUP|          5|           56|         74|   Y|                N|
|R1ZEDMCU6NYC8Q|          3|           26|         27|   Y|                N|
| RVBJM1LUI9X9K|          5|           27|         28|   Y|                N|
| R1KASHD9QSDMV|          4|           18|         21|   Y|                N|
|R1M3LATUI38MK0|          5|           46|         50|   Y|                N|
|R20JZQ3AQVYNAU|          4|          103|        113|   Y|                N|
|R1EHXYTMBO4D28|          1|           56|         61|   Y|     

497

In [ ]:
paid_review_no = filter_vine_df2.filter(filter_vine_df2["vine"]=="N")

In [ ]:
paid_review_no.show()
paid_review_no.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|                Y|
| RYA9ESIATW4EC|          1|           23|         27|   N|     

120863

In [ ]:
filter_vine_df2.count()

121360

In [ ]:
five_star_df = filter_vine_df2.filter(filter_vine_df2["star_rating"]=="5")
five_star_df.show()
five_star_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|                Y|
|R15RF9IWXWX3IH|          5|           25|         31|   N|                N|
|R1GNMWLUMCOC6V|          5|           21|         23|   N|                N|
|R2SLKTWAJ7BR9K|          5|           33|         36|   N|                Y|
|R1NKKJ720BC2VM|          5|           25|         28|   N|     

74690

In [ ]:
five_star_paid_review_yes = paid_review_yes.filter(paid_review_yes["star_rating"]=="5")
five_star_paid_review_yes.count()

220

In [ ]:
five_star_paid_review_no = paid_review_no.filter(paid_review_yes["star_rating"]=="5")
five_star_paid_review_no.count()

74470

In [ ]:
percentage_five_star_reviews_paid_yes = five_star_paid_review_yes.count()/  paid_review_yes.count()*100
percentage_five_star_reviews_paid_yes




44.26559356136821

In [ ]:
percentage_five_star_reviews_paid_no = five_star_paid_review_no.count()/ paid_review_no.count()*100
percentage_five_star_reviews_paid_no

61.615217229425056